In [2]:
!pip install --upgrade numpy==1.17.2

# Imports

In [13]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib. pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib. dates as mandates
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model
import tensorflow as tf

# Reading Dataset

In [14]:
dataset=pd.read_csv('../input/netflix-stock-price-prediction/NFLX.csv')
dataset.head()

Checking for null values: 

In [17]:
print("Dataframe Shape:", dataset.shape)
print("Null Value Present:", dataset.isnull().values.any())

# Visualization

In [19]:
import seaborn as sns

In [22]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.lineplot(data=dataset, x="Date", y="Adj Close")

In [23]:
y_data=dataset['Adj Close']
y_data.head()

In [25]:
features = ['Open', 'High', 'Low', 'Volume']

In [27]:
dataset_copy=dataset

# Scaling

In [28]:
scaler = MinMaxScaler()
feature_transform = scaler.fit_transform(dataset[features])
feature_transform= pd.DataFrame(columns=features, data=feature_transform, index=dataset.index)
feature_transform.head()

# Test - Train Split

In [30]:
timesplit= TimeSeriesSplit(n_splits=10)
for train_index, test_index in timesplit.split(feature_transform):
        X_train, X_test = feature_transform[:len(train_index)], feature_transform[len(train_index): (len(train_index)+len(test_index))]
        y_train, y_test = y_data[:len(train_index)].values.ravel(), y_data[len(train_index): (len(train_index)+len(test_index))].values.ravel()

In [31]:
trainX =np.array(X_train)
testX =np.array(X_test)
X_train = trainX.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = testX.reshape(X_test.shape[0], 1, X_test.shape[1])

# Modeling

In [32]:
#Building the LSTM Model
lstm = Sequential()
lstm.add(LSTM(32, input_shape=(1, trainX.shape[1]), activation='relu', return_sequences=False))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')
plot_model(lstm, show_shapes=True, show_layer_names=True)

In [33]:
#Model Training
history=lstm.fit(X_train, y_train, epochs=100, batch_size=8, verbose=1, shuffle=False)

In [38]:
#LSTM Prediction
y_pred= lstm.predict(X_test)

In [37]:
#Predicted vs True Adj Close Value – LSTM
plt.plot(y_test, label='True Value')
plt.plot(y_pred, label='LSTM Value')
plt.title("Prediction by LSTM")
plt.xlabel('Time Scale')
plt.ylabel('Scaled USD')
plt.legend()
plt.show()